# Parsing and generating robot descriptions from templates

## Parsing XACRO files

`.xacro` can also be parsed and inspected using the `pcg_gazebo_pkgs` library.

For this example please clone the `universal_robot` repository into you `catkin` workspace

```
cd $HOME/catkin_ws/src
git clone https://github.com/ros-industrial/universal_robot.git
```

and then build and source the catkin workspace.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from pcg_gazebo.simulation import SimulationModel
from pcg_gazebo.parsers import parse_xacro

In [ ]:
import os
import rospkg
try:
    pkg_path = rospkg.RosPack().get_path('ur_description')
    UR5_AVAILABLE = True
except rospkg.ResourceNotFound:
    UR5_AVAILABLE = False
    print('ur_description package not available')
    
if UR5_AVAILABLE:
    urdf_filename = os.path.join(pkg_path, 'urdf', 'ur5_robot.urdf.xacro')

Load the URDF model as a `SimulationModel`.

In [ ]:
if UR5_AVAILABLE:
    model = SimulationModel.from_urdf(parse_xacro(urdf_filename))

In [ ]:
if UR5_AVAILABLE:
    print('Links: ', model.link_names)

In [ ]:
if UR5_AVAILABLE:
    print('Joints: ', model.joint_names)

In [ ]:
if UR5_AVAILABLE:
    model.show(mesh_type='collision')

![sim_ur5_collision](images/sim_ur5_collision.png)

In [ ]:
if UR5_AVAILABLE:
    model.show(mesh_type='visual')

![sim_ur5_visual](images/sim_ur5_visual.png)

## Parsing Jinja templates to generate SDF robot descriptions 

Jinja is a powerful templating engine for Python. It can be extended with new functions and offers data structures such as dictionaries to be used inside the template.

In this example, the **kobuki** model was rewritten as a Jinja template (see below) to generate an SDF model for the robot. 

The extensions for the Jinja engines in the `pcg_gazebo` include 

* path completion using `$(find pkg)`, `package://`, `file://` 
* path completion for relative paths using `{% 'relative/path'|find_file %}` 
* macro functions used, for example, to compute moments of inertia for geometric primitives

In [ ]:
from pcg_gazebo.simulation import SimulationModel
from pcg_gazebo.parsers import parse_sdf
from pcg_gazebo.utils import process_jinja_template

Open and process the Jinja template.

The output XML is provided as a `string`.

In [ ]:
import os
import rospkg
jinja_template = os.path.abspath(
    os.path.join('robot_description', 'kobuki', 'sdf', 'kobuki.sdf.jinja'))
print('Jinja template = {}'.format(jinja_template))
output_xml = process_jinja_template(
    jinja_template,
    parameters=dict(robot_namespace='kobuki'))

Generate the SDF object for the XML output.

In [ ]:
sdf = parse_sdf(output_xml)
print(sdf.models)

Generate a model from the **kobuki** SDF data.

In [ ]:
model = SimulationModel.from_sdf(sdf.models[0])

Visualize the model's collision and visual geometries

In [ ]:
model.show(mesh_type='visual')

In [ ]:
model.show(mesh_type='collision')

In [ ]:
from pcg_gazebo.generators.creators import create_models_from_config
from pcg_gazebo.task_manager import Server, get_rostopic_list

# Start an empty world Gazebo simulation
server = Server()
server.create_simulation('default')
simulation = server.get_simulation('default')
simulation.create_gazebo_empty_world_task()
simulation.run_task('gazebo')

# Create a Gazebo proxy
gazebo_proxy = simulation.get_gazebo_proxy()

Spawn the **kobuki** model.

In [ ]:
model.spawn(gazebo_proxy=gazebo_proxy)

List all ROS topics

In [ ]:
print('ROS topics: ', get_rostopic_list(gazebo_proxy.ros_config.ros_master_uri))

In [ ]:
simulation.init_task(
    name='command',
    command='rostopic pub /kobuki/cmd_vel geometry_msgs/Twist "{linear: {x: {vx}, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: {vtheta}}}"',
    has_gazebo=False,
    params=dict(
        vx=0.3,
        vtheta=0.4))
simulation.run_task('command')

![sim_kobuki_cmd](images/sim_kobuki_cmd.png)

In [ ]:
#simulation.kill_all_tasks()